In [ ]:
#Upload your kaggle API json file:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vamsipvs","key":"11b9c77decceaea998a95ec44402006f"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ 
!ls ~/.kaggle

! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list &> /dev/null

kaggle.json


In [ ]:
!kaggle datasets download -d ashirwadsangwan/imdb-dataset

100% 1.43G/1.44G [00:36<00:00, 25.5MB/s]
100% 1.44G/1.44G [00:36<00:00, 42.6MB/s]


In [ ]:
! mkdir -p dataset 
! unzip imdb-dataset.zip -d dataset &> /dev/null

In [ ]:
!sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [679 kB]
Get:

In [ ]:
!sudo apt-get install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 76 not upgraded.


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
## importing libraries
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init("spark-3.1.2-bin-hadoop3.2")# SPARK_HOME
from pyspark.sql import SparkSession
import pyspark.sql.functions  as F
from pyspark.sql.functions import explode,split
from pyspark import SparkContext

In [ ]:
sc=SparkContext()
session = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.functions import col, size, collect_set
import pyspark.sql.functions as F
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
import pickle
from tqdm.auto import tqdm
import time
import itertools
from collections import defaultdict
import functools
import operator
from pyspark.ml.fpm import FPGrowth
import time
#from collections import defaultdict
from google.colab import drive, files
import os

In [ ]:
actors = session.read.csv("/content/dataset/name.basics.tsv/name.basics.tsv", header=True, inferSchema=True,sep=r'\t')
actors.show(10, False)

+---------+---------------+---------+---------+------------------------------+---------------------------------------+
|nconst   |primaryName    |birthYear|deathYear|primaryProfession             |knownForTitles                         |
+---------+---------------+---------+---------+------------------------------+---------------------------------------+
|nm0000001|Fred Astaire   |1899     |1987     |soundtrack,actor,miscellaneous|tt0050419,tt0053137,tt0072308,tt0043044|
|nm0000002|Lauren Bacall  |1924     |2014     |actress,soundtrack            |tt0071877,tt0117057,tt0038355,tt0037382|
|nm0000003|Brigitte Bardot|1934     |\N       |actress,soundtrack,producer   |tt0054452,tt0049189,tt0059956,tt0057345|
|nm0000004|John Belushi   |1949     |1982     |actor,writer,soundtrack       |tt0077975,tt0072562,tt0080455,tt0078723|
|nm0000005|Ingmar Bergman |1918     |2007     |writer,director,actor         |tt0069467,tt0050976,tt0083922,tt0050986|
|nm0000006|Ingrid Bergman |1915     |1982     |a

In [ ]:
actors = ( actors
          .select(col('nconst').alias('actor_id'), col('primaryName'), col('primaryProfession') )
          .where( (col('actor_id').isNotNull()) & (col('primaryName').isNotNull()) )
          .filter( (col('actor_id') != '\\N') | (col('primaryName') != '\\N') )
          )

In [ ]:
actors.createOrReplaceTempView('actors')

query = """
  SELECT *
  FROM actors
  WHERE (primaryProfession LIKE '%actor%') OR
        (primaryProfession LIKE '%actress%')
"""
actors = session.sql(query)
actors.show(10, False) 

+---------+---------------+------------------------------+
|actor_id |primaryName    |primaryProfession             |
+---------+---------------+------------------------------+
|nm0000001|Fred Astaire   |soundtrack,actor,miscellaneous|
|nm0000002|Lauren Bacall  |actress,soundtrack            |
|nm0000003|Brigitte Bardot|actress,soundtrack,producer   |
|nm0000004|John Belushi   |actor,writer,soundtrack       |
|nm0000005|Ingmar Bergman |writer,director,actor         |
|nm0000006|Ingrid Bergman |actress,soundtrack,producer   |
|nm0000007|Humphrey Bogart|actor,soundtrack,producer     |
|nm0000008|Marlon Brando  |actor,soundtrack,director     |
|nm0000009|Richard Burton |actor,producer,soundtrack     |
|nm0000010|James Cagney   |actor,soundtrack,director     |
+---------+---------------+------------------------------+
only showing top 10 rows



In [ ]:
print('Number of actors:', actors.count())

Number of actors: 3624970


In [ ]:
movies_to_actors = session.read.csv('/content/dataset/title.principals.tsv/title.principals.tsv',header=True, inferSchema=True, sep='\t')
movies_to_actors.show(10, False)

+---------+--------+---------+---------------+-----------------------+-----------+
|tconst   |ordering|nconst   |category       |job                    |characters |
+---------+--------+---------+---------------+-----------------------+-----------+
|tt0000001|1       |nm1588970|self           |\N                     |["Herself"]|
|tt0000001|2       |nm0005690|director       |\N                     |\N         |
|tt0000001|3       |nm0374658|cinematographer|director of photography|\N         |
|tt0000002|1       |nm0721526|director       |\N                     |\N         |
|tt0000002|2       |nm1335271|composer       |\N                     |\N         |
|tt0000003|1       |nm0721526|director       |\N                     |\N         |
|tt0000003|2       |nm5442194|producer       |producer               |\N         |
|tt0000003|3       |nm1335271|composer       |\N                     |\N         |
|tt0000003|4       |nm5442200|editor         |\N                     |\N         |
|tt0

In [ ]:
movies_to_actors = ( movies_to_actors
          .select(col('tconst').alias('movie_id'), col('nconst').alias('actor_id'), col('category'), col('job') )
          .where( (col('movie_id').isNotNull()) & (col('actor_id').isNotNull()) ) 
          .filter( (col('movie_id') != '\\N') | (col('actor_id') != '\\N'))
          )

In [ ]:
movies_to_actors.createOrReplaceTempView('movies_to_actors')
query = """
  SELECT *
  FROM movies_to_actors
  WHERE (category LIKE '%actor%') OR
        (category LIKE '%actress%') OR
        (job LIKE '%actor%') OR
        (job LIKE '%actress%')
"""
movies_to_actors = session.sql(query)
movies_to_actors.show(10, False) 

+---------+---------+--------+---+
|movie_id |actor_id |category|job|
+---------+---------+--------+---+
|tt0000005|nm0443482|actor   |\N |
|tt0000005|nm0653042|actor   |\N |
|tt0000007|nm0179163|actor   |\N |
|tt0000007|nm0183947|actor   |\N |
|tt0000008|nm0653028|actor   |\N |
|tt0000009|nm0063086|actress |\N |
|tt0000009|nm0183823|actor   |\N |
|tt0000009|nm1309758|actor   |\N |
|tt0000011|nm3692297|actor   |\N |
|tt0000014|nm0166380|actor   |\N |
+---------+---------+--------+---+
only showing top 10 rows



In [ ]:
print('Number of correspondences:', movies_to_actors.count())

Number of correspondences: 14818830


In [ ]:
movies = session.read.csv('/content/dataset/title.basics.tsv/title.basics.tsv',header=True, inferSchema=True, sep='\t')
movies.show(10, False)

+---------+---------+-------------------------------------------+-------------------------------------------+-------+---------+-------+--------------+------------------------+
|tconst   |titleType|primaryTitle                               |originalTitle                              |isAdult|startYear|endYear|runtimeMinutes|genres                  |
+---------+---------+-------------------------------------------+-------------------------------------------+-------+---------+-------+--------------+------------------------+
|tt0000001|short    |Carmencita                                 |Carmencita                                 |0      |1894     |\N     |1             |Documentary,Short       |
|tt0000002|short    |Le clown et ses chiens                     |Le clown et ses chiens                     |0      |1892     |\N     |5             |Animation,Short         |
|tt0000003|short    |Pauvre Pierrot                             |Pauvre Pierrot                             |0      |189

In [ ]:
movies = ( movies
          .select(col('tconst').alias('movie_id'), col('primaryTitle'), col('genres') )
          .where( (col('movie_id').isNotNull()) & (col('primaryTitle').isNotNull()) ) 
          .filter( (col('movie_id') != '\\N') | (col('primaryTitle') != '\\N'))
          )
movies.show(10, False)

+---------+-------------------------------------------+------------------------+
|movie_id |primaryTitle                               |genres                  |
+---------+-------------------------------------------+------------------------+
|tt0000001|Carmencita                                 |Documentary,Short       |
|tt0000002|Le clown et ses chiens                     |Animation,Short         |
|tt0000003|Pauvre Pierrot                             |Animation,Comedy,Romance|
|tt0000004|Un bon bock                                |Animation,Short         |
|tt0000005|Blacksmith Scene                           |Comedy,Short            |
|tt0000006|Chinese Opium Den                          |Short                   |
|tt0000007|Corbett and Courtney Before the Kinetograph|Short,Sport             |
|tt0000008|Edison Kinetoscopic Record of a Sneeze     |Documentary,Short       |
|tt0000009|Miss Jerry                                 |Romance                 |
|tt0000010|Exiting the Facto

In [ ]:
print('Number of movies:',movies.count() )

Number of movies: 6321302


In [ ]:
actors = actors.repartition(10)
movies = movies.repartition(10)
movies_to_actors = movies_to_actors.repartition(10)

In [ ]:
join_data = ( movies_to_actors
 .join(movies, movies_to_actors.movie_id == movies.movie_id,how='inner')
 .join(actors, movies_to_actors.actor_id == actors.actor_id, how='inner')
 .select(movies_to_actors.movie_id, col('primaryTitle').alias('movie_title'),movies_to_actors.actor_id, col('primaryName').alias('actor_name'))
 .orderBy(movies.movie_id) )

join_data.show(10, False)

+---------+-------------------------------------------+---------+-----------------+
|movie_id |movie_title                                |actor_id |actor_name       |
+---------+-------------------------------------------+---------+-----------------+
|tt0000005|Blacksmith Scene                           |nm0443482|Charles Kayser   |
|tt0000005|Blacksmith Scene                           |nm0653042|John Ott         |
|tt0000007|Corbett and Courtney Before the Kinetograph|nm0179163|James J. Corbett |
|tt0000007|Corbett and Courtney Before the Kinetograph|nm0183947|Peter Courtney   |
|tt0000009|Miss Jerry                                 |nm0183823|William Courtenay|
|tt0000009|Miss Jerry                                 |nm0063086|Blanche Bayliss  |
|tt0000009|Miss Jerry                                 |nm1309758|Chauncey Depew   |
|tt0000014|The Sprinkler Sprinkled                    |nm0166380|François Clerc   |
|tt0000014|The Sprinkler Sprinkled                    |nm0244989|Benoît Duva

In [ ]:
join_data.cache()

DataFrame[movie_id: string, movie_title: string, actor_id: string, actor_name: string]

In [ ]:
baskets_movies = (join_data
                  .groupBy('movie_id')
                  .agg(F.collect_set('actor_name').alias('actors_in_movie'))
                  .sort(col("movie_id").asc())
                  )

In [ ]:
baskets_movies.show(10,False)

+---------+----------------------------------------------------+
|movie_id |actors_in_movie                                     |
+---------+----------------------------------------------------+
|tt0000005|[Charles Kayser, John Ott]                          |
|tt0000007|[James J. Corbett, Peter Courtney]                  |
|tt0000009|[William Courtenay, Chauncey Depew, Blanche Bayliss]|
|tt0000014|[François Clerc, Benoît Duval]                      |
|tt0000018|[Delaware]                                          |
|tt0000026|[Félicien Trewey, Antoine Féraud]                   |
|tt0000032|[Ancion]                                            |
|tt0000036|[Joseph Jefferson]                                  |
|tt0000061|[Alhambra Girls]                                    |
|tt0000063|[Georges Méliès]                                    |
+---------+----------------------------------------------------+
only showing top 10 rows



In [ ]:
baskets_movies.cache()
movies.cache()
join_data.cache()

DataFrame[movie_id: string, movie_title: string, actor_id: string, actor_name: string]

In [ ]:
moviesRDD = ( movies
             .where(col('genres').isNotNull())
             .rdd
             .map(lambda x: x[2].split(sep=',') ) 
             .flatMap(lambda x: x) 
             .distinct()
             .collect() )
display(moviesRDD)

['Talk-Show',
 'Music',
 'Animation',
 'War',
 'Sci-Fi',
 'Horror',
 'News',
 'Musical',
 '\\N',
 'Crime',
 'Reality-TV',
 'Action',
 'Family',
 'Romance',
 'Documentary',
 'Thriller',
 'Western',
 'History',
 'Drama',
 'Adult',
 'Short',
 'Fantasy',
 'Sport',
 'Game-Show',
 'Biography',
 'Comedy',
 'Adventure',
 'Mystery',
 'Film-Noir']

In [ ]:
movies.createOrReplaceTempView('movies')
baskets_movies.createOrReplaceTempView('baskets_movies')

In [ ]:
query = """
  SELECT baskets_movies.movie_id AS movie_id, actors_in_movie
  FROM baskets_movies INNER JOIN movies 
    ON baskets_movies.movie_id = movies.movie_id
  WHERE genres LIKE '%Musical%'
"""
baskets_movies_reduced = session.sql(query)

In [ ]:
print('Number of baskets:',len(baskets_movies_reduced.select(col('movie_id')).rdd.collect()) )

Number of baskets: 35111


In [ ]:
class APriori():

  def __init__(self, 
               data, 
               min_support, 
               calculate_association_rules=True):
    self.data = data
    self.min_support = min_support
    self.get_association_rules = [] 
    self.calculate_association_rules = calculate_association_rules


  def baskets(self):
  
    baskets = (self.data
               .rdd.map(tuple)
               .map(lambda a: a[1])
               .collect() )
    return baskets
  

  def frequent_singletons(self, counter=False):  
    data = self.data.rdd.map(tuple)
    min_support = self.min_support
    n_baskets = len(self.data.collect())
    frequent_singletons_counter = (data
                          .flatMap(lambda a: a[1])
                          .map(lambda a: (a,1))
                          .reduceByKey(lambda a,b: a+b)
                          .filter(lambda a: a[1] > min_support*n_baskets ) ) 
    frequent_singletons = ( frequent_singletons_counter
                           .map(lambda a: a[0])
                           .collect() )
    if counter:
      return frequent_singletons_counter
    else:
      return frequent_singletons


  def candidate_itemsets(self,frequent_itemsets, i):

    candidates = []
    for itemset in itertools.product(frequent_itemsets,frequent_itemsets):
      if len(set(functools.reduce(operator.iconcat, itemset, []))) == i:
        candidates.append(set(functools.reduce(operator.iconcat, itemset, [])))

    return candidates


  def get_frequent_itemsets(self):
    
    baskets = self.baskets()
    counter = defaultdict(lambda: 0)
    final_frequent_itemsets = ( self.frequent_singletons(counter=True)
                                .map(lambda a: ([a[0]],a[1]))
                                .collect() )
    
    for candidate in tqdm(itertools.combinations(self.frequent_singletons(),2),
                        total=sum(1 for _ in itertools.combinations(self.frequent_singletons(),2)),
                        desc = 'Retrieving frequent itemsets with {} items'.format(2)):

      for basket in baskets:
        if set(candidate).issubset(basket):
          counter[frozenset(candidate)] += 1

    frequent_itemsets = []

    for key, val in counter.items():
      if counter[key] >= self.min_support*len(counter): 
        frequent_itemsets.append(list(key))
        final_frequent_itemsets.append((list(key),val))
    
    i=3
    while frequent_itemsets:
      counter = defaultdict(lambda: 0)

      for candidate in tqdm( set(tuple(sorted(c)) for c in self.candidate_itemsets(frequent_itemsets,i) ),
                            'Retrieving frequent itemsets with {} items'.format(i)):

        for basket in baskets:
          if set(candidate).issubset(basket):
            counter[frozenset(candidate)] += 1
  
      frequent_itemsets = []

      for key, val in counter.items():
        if counter[key] >= self.min_support*len(counter): 
          frequent_itemsets.append(list(key))
          final_frequent_itemsets.append((list(key),val))

      i+=1

    return sc.parallelize( [ (item) for item in final_frequent_itemsets ] ).toDF([*['items','freq']])


In [ ]:
start = time.time()

apriori = APriori(data=baskets_movies_reduced, min_support= 0.0006)
frequent_itemsets = apriori.get_frequent_itemsets()


end = time.time()

print("Runtime of the program is {} hours".format((round((end - start)/60/60, 2))))


Runtime of the program is 1.29 hours


In [ ]:
frequent_itemsets = frequent_itemsets.orderBy(col('freq').desc())
frequent_itemsets.show(20,False)

+----------------------------------------------------------------------+----+
|items                                                                 |freq|
+----------------------------------------------------------------------+----+
|[Thomas Sanders]                                                      |1887|
|[Manuela do Monte]                                                    |472 |
|[Rayssa Chaddad, Manuela do Monte]                                    |471 |
|[Felipe Cavalcanti, Rayssa Chaddad, Manuela do Monte]                 |471 |
|[Manuela do Monte, Giovanna Grigio]                                   |471 |
|[Felipe Cavalcanti, Manuela do Monte]                                 |471 |
|[Felipe Cavalcanti]                                                   |471 |
|[Felipe Cavalcanti, Giovanna Grigio]                                  |471 |
|[Felipe Cavalcanti, Manuela do Monte, Giovanna Grigio]                |471 |
|[Felipe Cavalcanti, Rayssa Chaddad, Giovanna Grigio]           